In [ ]:
#@title 🛠️ **Cell 1: Trinity Infrastructure with Dynamic PyPI Resolution** { display-mode: "form" }

import os
import subprocess
import sys
import time
import json
import requests
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import urllib.parse

PROJECT_ROOT = Path('/content/TrinityUI')
WEBUI_ROOT = Path('/content')
TRINITY_REPO_URL = 'https://github.com/remphanstar/TrinityUI.git'
TRINITY_BRANCH = 'main'

def run_setup_command(command, show_output=True, ignore_errors=False):
    """Runs a shell command and streams its output live."""
    start_time = time.time()
    print(f"🚀 Executing: {command}")
    process = subprocess.Popen(
        command,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        encoding='utf-8',
        errors='replace'
    )

    output_lines = []
    for line in iter(process.stdout.readline, ''):
        output_lines.append(line)
        if show_output:
            sys.stdout.write(f"  > {line}")
            sys.stdout.flush()

    return_code = process.wait()
    duration = time.time() - start_time

    if return_code != 0 and not ignore_errors:
        print(f"❌ Command failed with exit code {return_code}")
        raise subprocess.CalledProcessError(return_code, command)

    if return_code == 0:
        print(f"✅ Command successful. ({duration:.2f}s)\n")
        return True, output_lines
    else:
        print(f"⚠️ Command returned non-zero exit code: {return_code} ({duration:.2f}s)\n")
        return False, output_lines

def get_pypi_package_url(package_name, version, python_version="cp310"):
    """Dynamically resolve PyPI package URLs"""
    try:
        # Query PyPI JSON API
        api_url = f"https://pypi.org/pypi/{package_name}/{version}/json"
        response = requests.get(api_url, timeout=10)

        if response.status_code == 200:
            data = response.json()
            files = data.get('urls', [])

            # Look for compatible wheel
            for file_info in files:
                filename = file_info.get('filename', '')
                if (filename.endswith('.whl') and
                    python_version in filename and
                    'linux_x86_64' in filename):
                    return file_info.get('url')

            # Fallback to any wheel
            for file_info in files:
                filename = file_info.get('filename', '')
                if filename.endswith('.whl'):
                    return file_info.get('url')

            # Fallback to source distribution
            for file_info in files:
                filename = file_info.get('filename', '')
                if filename.endswith('.tar.gz'):
                    return file_info.get('url')

        print(f"⚠️ Could not find download URL for {package_name}=={version}")
        return None

    except Exception as e:
        print(f"⚠️ Error resolving {package_name}: {e}")
        return None

def validate_url(url):
    """Validate that a URL is accessible"""
    try:
        response = requests.head(url, timeout=10, allow_redirects=True)
        return response.status_code == 200
    except:
        return False

def setup_aria2c_for_bulk_install():
    """Configure aria2c for massive parallel downloads"""
    config = {
        'max-connection-per-server': '8',  # Reduced to be more PyPI-friendly
        'max-concurrent-downloads': '8',
        'split': '8',
        'min-split-size': '1M',
        'max-overall-download-limit': '0',
        'max-download-limit': '0',
        'continue': 'true',
        'auto-file-renaming': 'false',
        'allow-overwrite': 'true',
        'retry-wait': '3',
        'max-tries': '5',
        'file-allocation': 'none',
        'check-integrity': 'true',
        'user-agent': 'aria2/1.36.0'
    }

    config_file = Path('/tmp/aria2c_bulk.conf')
    with open(config_file, 'w') as f:
        for key, value in config.items():
            f.write(f"{key}={value}\n")

    print(f"✅ Aria2c config created with PyPI-friendly settings")
    return str(config_file)

def get_webui_requirements():
    """Get all WebUI requirements for batch download"""
    webuis = {
        'A1111': {
            'repo': 'https://github.com/AUTOMATIC1111/stable-diffusion-webui.git',
            'python': '3.10',
            'key_packages': [
                ('torch', '2.1.2+cu121'),
                ('torchvision', '0.16.2+cu121'),
                ('torchaudio', '2.1.2+cu121'),
                ('transformers', '4.30.2'),
                ('safetensors', '0.4.2')
            ]
        },
        'Forge': {
            'repo': 'https://github.com/lllyasviel/stable-diffusion-webui-forge.git',
            'python': '3.11',
            'key_packages': [
                ('torch', '2.3.1+cu121'),
                ('torchvision', '0.18.1+cu121'),
                ('transformers', '4.46.1')
            ]
        },
        'ComfyUI': {
            'repo': 'https://github.com/comfyanonymous/ComfyUI.git',
            'python': '3.10',
            'key_packages': [
                ('torch', '2.1.2+cu121'),
                ('torchvision', '0.16.2+cu121'),
                ('torchaudio', '2.1.2+cu121'),
                ('transformers', '4.37.2'),
                ('safetensors', '0.4.2')
            ]
        }
    }
    return webuis

def create_bulk_download_list():
    """Create aria2c input file with validated URLs"""
    requirements = get_webui_requirements()
    download_list = []
    base_cache_dir = Path('/tmp/webui_cache')
    base_cache_dir.mkdir(exist_ok=True)

    print(f"🔍 Resolving PyPI package URLs dynamically...")

    # PyTorch URLs (these are more stable)
    pytorch_wheels = {
        ('torch', '2.1.2+cu121', 'cp310'): 'https://download.pytorch.org/whl/cu121/torch-2.1.2%2Bcu121-cp310-cp310-linux_x86_64.whl',
        ('torchvision', '0.16.2+cu121', 'cp310'): 'https://download.pytorch.org/whl/cu121/torchvision-0.16.2%2Bcu121-cp310-cp310-linux_x86_64.whl',
        ('torchaudio', '2.1.2+cu121', 'cp310'): 'https://download.pytorch.org/whl/cu121/torchaudio-2.1.2%2Bcu121-cp310-cp310-linux_x86_64.whl',
        ('torch', '2.3.1+cu121', 'cp311'): 'https://download.pytorch.org/whl/cu121/torch-2.3.1%2Bcu121-cp311-cp311-linux_x86_64.whl',
        ('torchvision', '0.18.1+cu121', 'cp311'): 'https://download.pytorch.org/whl/cu121/torchvision-0.18.1%2Bcu121-cp311-cp311-linux_x86_64.whl'
    }

    valid_downloads = 0
    total_packages = 0

    for webui, config in requirements.items():
        webui_cache = base_cache_dir / webui
        webui_cache.mkdir(exist_ok=True)

        python_tag = f"cp{config['python'].replace('.', '')}"

        for package_name, version in config['key_packages']:
            total_packages += 1

            # Check if it's a PyTorch package
            pytorch_key = (package_name, version, python_tag)
            if pytorch_key in pytorch_wheels:
                url = pytorch_wheels[pytorch_key]
                filename = url.split('/')[-1].replace('%2B', '+')

                print(f"   📦 {package_name}=={version} (PyTorch) -> {filename}")

                download_list.append({
                    'url': url,
                    'dir': str(webui_cache),
                    'filename': filename,
                    'webui': webui,
                    'package': package_name
                })
                valid_downloads += 1
                continue

            # For other packages, resolve dynamically
            clean_version = version.replace('+cu121', '').replace('+cu118', '')
            url = get_pypi_package_url(package_name, clean_version, python_tag)

            if url and validate_url(url):
                filename = url.split('/')[-1]
                print(f"   📦 {package_name}=={clean_version} (PyPI) -> {filename}")

                download_list.append({
                    'url': url,
                    'dir': str(webui_cache),
                    'filename': filename,
                    'webui': webui,
                    'package': package_name
                })
                valid_downloads += 1
            else:
                print(f"   ⚠️ {package_name}=={clean_version} - will install via pip fallback")

    print(f"📊 Resolved {valid_downloads}/{total_packages} package URLs for aria2c download")
    return download_list

def parallel_webui_install():
    """Install all WebUIs with aria2c acceleration and pip fallback"""
    print("\n" + "="*80)
    print("🚀 STARTING OPTIMIZED WEBUI INSTALLATION")
    print("="*80)

    config_file = setup_aria2c_for_bulk_install()
    download_list = create_bulk_download_list()

    if not download_list:
        print("⚠️ No packages available for aria2c, will use pip only")
        return True  # Continue with pip-only installation

    # Create input file for aria2c
    input_file = Path('/tmp/bulk_webui_downloads.txt')
    with open(input_file, 'w') as f:
        for item in download_list:
            f.write(f"{item['url']}\n")
            f.write(f" dir={item['dir']}\n")
            f.write(f" out={item['filename']}\n\n")

    print(f"⚡ Starting aria2c download of {len(download_list)} validated packages...")

    start_time = time.time()

    # Run aria2c with config
    cmd = [
        'aria2c',
        f'--conf-path={config_file}',
        f'--input-file={str(input_file)}',
        '--summary-interval=10',
        '--download-result=full'
    ]

    try:
        process = subprocess.Popen(
            cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
            text=True, bufsize=1
        )

        for line in iter(process.stdout.readline, ''):
            if any(keyword in line for keyword in ['Download complete', 'ERROR', 'NOTICE', '[#', 'GID']):
                print(f"  > {line.strip()}")

        return_code = process.wait()
        download_time = time.time() - start_time

        if return_code == 0:
            print(f"\n✅ Aria2c downloads completed in {download_time:.2f}s!")
        else:
            print(f"\n⚠️ Aria2c had some failures (exit code {return_code})")
            print("Will use pip fallback for failed packages")

        # Check what was actually downloaded
        successful_downloads = 0
        for item in download_list:
            file_path = Path(item['dir']) / item['filename']
            if file_path.exists() and file_path.stat().st_size > 0:
                successful_downloads += 1

        print(f"📊 Successfully downloaded {successful_downloads}/{len(download_list)} packages")

        return True  # Continue regardless, pip will handle missing packages

    except Exception as e:
        print(f"⚠️ Aria2c execution error: {e}")
        print("Will continue with pip-only installation")
        return True

def clone_all_webuis():
    """Clone all WebUI repositories in parallel"""
    print("\n🔄 Cloning WebUI repositories...")

    requirements = get_webui_requirements()

    def clone_webui(webui_name, config):
        webui_path = WEBUI_ROOT / webui_name
        if not webui_path.exists():
            try:
                cmd = f"git clone --depth 1 {config['repo']} {webui_path}"
                result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=300)
                if result.returncode == 0:
                    print(f"   ✅ {webui_name} cloned successfully")
                    return True
                else:
                    print(f"   ❌ {webui_name} clone failed: {result.stderr[:100]}")
                    return False
            except Exception as e:
                print(f"   ❌ {webui_name} clone exception: {e}")
                return False
        else:
            print(f"   ✅ {webui_name} already exists")
            return True

    # Clone in parallel
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = [executor.submit(clone_webui, name, config) for name, config in requirements.items()]
        results = [future.result() for future in futures]

    success_count = sum(results)
    print(f"📊 Successfully prepared {success_count}/{len(requirements)} WebUIs")
    return success_count >= 1  # Success if at least one WebUI is ready

def create_optimized_venvs():
    """Create virtual environments with optimized pip settings"""
    print("\n🔧 Creating optimized virtual environments...")

    requirements = get_webui_requirements()
    python_paths = {
        '3.10': '/usr/bin/python3.10',
        '3.11': '/usr/bin/python3.11'
    }

    # Setup global pip optimizations
    pip_config_dir = Path.home() / '.pip'
    pip_config_dir.mkdir(exist_ok=True)

    pip_config = pip_config_dir / 'pip.conf'
    pip_config_content = """[global]
timeout = 60
retries = 3
trusted-host = pypi.org
               pypi.python.org
               files.pythonhosted.org
cache-dir = /tmp/pip-cache
progress-bar = on

[install]
compile = false
"""
    pip_config.write_text(pip_config_content)

    cache_dir = Path('/tmp/pip-cache')
    cache_dir.mkdir(exist_ok=True)

    def create_venv(webui_name, config):
        webui_path = WEBUI_ROOT / webui_name
        venv_path = webui_path / 'venv'

        if not webui_path.exists():
            print(f"   ⚠️ {webui_name} repository not found, skipping")
            return False

        if venv_path.exists():
            print(f"   ✅ {webui_name} venv already exists")
            return True

        python_exe = python_paths.get(config['python'])
        if not python_exe:
            print(f"   ❌ Python {config['python']} not found for {webui_name}")
            return False

        try:
            # Create venv
            cmd = f"{python_exe} -m venv {venv_path}"
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=120)
            if result.returncode != 0:
                print(f"   ❌ {webui_name} venv creation failed")
                return False

            # Upgrade pip with optimizations
            venv_pip = venv_path / 'bin' / 'pip'
            pip_cmd = f'"{venv_pip}" install --upgrade pip setuptools wheel --cache-dir=/tmp/pip-cache'
            subprocess.run(pip_cmd, shell=True, capture_output=True, timeout=120)

            print(f"   ✅ {webui_name} venv created with Python {config['python']}")
            return True

        except Exception as e:
            print(f"   ❌ {webui_name} venv exception: {e}")
            return False

    # Create venvs in parallel
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = [executor.submit(create_venv, name, config) for name, config in requirements.items()]
        results = [future.result() for future in futures]

    success_count = sum(results)
    print(f"📊 Successfully created {success_count}/{len(requirements)} venvs")
    return success_count >= 1

print("🚀 Starting Trinity Infrastructure with Dynamic PyPI Resolution...")

# 1. Install system dependencies
print("\n[Step 1/5] Installing system dependencies...")
try:
    os.environ['DEBIAN_FRONTEND'] = 'noninteractive'
    success, output = run_setup_command("""
        apt-get update -qq &&
        apt-get install -y -qq aria2 git curl wget python3.10 python3.10-venv python3.11 python3.11-venv python3.10-dev python3.11-dev build-essential
    """)
    print("   ✅ System dependencies installed.")
except Exception as e:
    print(f"   ❌ FAILED to install dependencies: {e}")
    raise

# 2. Clone TrinityUI control repository
print(f"\n[Step 2/5] Setting up TrinityUI control repository...")
try:
    if (PROJECT_ROOT / '.git').is_dir():
        print("   - Updating existing repository...")
        run_setup_command(f"git -C {PROJECT_ROOT} fetch && git -C {PROJECT_ROOT} reset --hard origin/{TRINITY_BRANCH}")
    else:
        print("   - Cloning repository...")
        run_setup_command(f"git clone --depth 1 --branch {TRINITY_BRANCH} --single-branch {TRINITY_REPO_URL} {PROJECT_ROOT}")
    print(f"   ✅ TrinityUI repository ready.")
except Exception as e:
    print(f"   ❌ FAILED to setup repository: {e}")
    raise

# 3. Clone WebUI repositories
print(f"\n[Step 3/5] Setting up WebUI repositories...")
clone_success = clone_all_webuis()

# 4. Create optimized virtual environments
print(f"\n[Step 4/5] Creating optimized virtual environments...")
venv_success = create_optimized_venvs()

# 5. Aria2c optimized downloads with fallback
print(f"\n[Step 5/5] Optimized package downloads...")
download_success = parallel_webui_install()

# Summary
print("\n" + "="*80)
print("🎉 TRINITY OPTIMIZED INFRASTRUCTURE SUMMARY")
print("="*80)
print(f"✅ System dependencies: ✅ Installed")
print(f"✅ TrinityUI repository: ✅ Ready")
print(f"✅ WebUI repositories: {'✅ Ready' if clone_success else '⚠️ Partial'}")
print(f"✅ Virtual environments: {'✅ Created' if venv_success else '⚠️ Partial'}")
print(f"✅ Optimized downloads: {'✅ Completed' if download_success else '⚠️ Fallback mode'}")

overall_success = clone_success and venv_success

if overall_success:
    print(f"\n🎉 INFRASTRUCTURE READY - aria2c acceleration active where possible!")
    print("📊 Next steps:")
    print("   - Run Cell 2 for configuration")
    print("   - Dependencies will install with maximum speed")
    print("   - Fallback to pip for any aria2c failures")
else:
    print(f"\n⚠️ Basic setup completed with some optimizations")

# Set global variables
globals()['PROJECT_ROOT'] = PROJECT_ROOT
globals()['WEBUI_ROOT'] = WEBUI_ROOT
globals()['trinity_setup_success'] = True
globals()['optimized_install_ready'] = overall_success

print(f"\n🔄 NEXT: Run Cell 2 for configuration - infrastructure is optimized!")

In [ ]:
#@title 🎛️ **Cell 2: Trinity Configuration Hub** { display-mode: "form" }

import os
import sys
import importlib.util
from pathlib import Path
from datetime import datetime
import socket

def find_free_port(start_port=7860, max_attempts=20):
    """Find a free port starting from start_port"""
    for i in range(max_attempts):
        port = start_port + i
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('127.0.0.1', port))
                return port
            except OSError:
                continue
    return None

def log_to_unified(message, level="INFO", component="TRINITY-CELL-2"):
    """Log messages using the unified logging system from Cell 1"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_file = globals().get('LOG_FILE', Path.cwd() / 'trinity_unified.log')
    version = globals().get('VERSION', '1.1.0')
    log_entry = f"[{timestamp}] [v{version}] [{level}] [{component}] {message}\n"
    try:
        with open(log_file, 'a', encoding='utf-8') as f:
            f.write(log_entry)
    except Exception as e:
        print(f"Warning: Could not write to log file: {e}")
    if level in ["ERROR", "SUCCESS"] or "Trinity" in message:
        print(f"[{level}] {message}")

# Infrastructure Check
def check_trinity_infrastructure():
    if 'PROJECT_ROOT' not in globals() or not globals()['PROJECT_ROOT'].exists():
        return False, "PROJECT_ROOT not found. Please run Cell 1 first."
    return True, "Infrastructure OK"

infrastructure_ok, infrastructure_msg = check_trinity_infrastructure()
if not infrastructure_ok:
    print(f"❌ CRITICAL: {infrastructure_msg}")
    raise RuntimeError(infrastructure_msg)

# Main Execution
PROJECT_ROOT = globals()['PROJECT_ROOT']
sys.path.insert(0, str(PROJECT_ROOT))

log_to_unified("Preparing Trinity Configuration Hub...")

# Find a free port
config_port = find_free_port(7860, 20)
if config_port is None:
    log_to_unified("Could not find a free port, using fallback 7861", "WARNING")
    config_port = 7861
else:
    log_to_unified(f"Found free port for Config Hub: {config_port}", "SUCCESS")

os.environ['TRINITY_CONFIG_PORT'] = str(config_port)

# Load and execute the configuration hub script
try:
    config_script_path = PROJECT_ROOT / 'scripts' / 'configuration_hub_new.py'

    if config_script_path.exists():
        log_to_unified(f"Executing configuration hub from: {config_script_path}")

        # Use importlib to load the script as a module
        spec = importlib.util.spec_from_file_location("config_hub_module", config_script_path)
        config_hub_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(config_hub_module)

        # Explicitly call the function that launches the Gradio app
        if hasattr(config_hub_module, 'launch_trinity_configuration_hub'):
            log_to_unified("Launching Gradio Configuration Interface...")
            print("🎛️ Trinity Configuration Hub")
            print("=" * 50)
            print("🔧 Configure your WebUI settings in the interface below")
            print("📝 Select models, VAEs, ControlNets, and other options")
            print("💾 Click 'Save Configuration' when ready")
            print("=" * 50)

            # This function contains the launch() call with share=True
            config_hub_module.launch_trinity_configuration_hub()
        else:
            raise RuntimeError("Could not find 'launch_trinity_configuration_hub' function in the script.")

    else:
        log_to_unified("Configuration Hub script (configuration_hub_new.py) not found!", "ERROR")
        print("❌ ERROR: scripts/configuration_hub_new.py not found.")

except Exception as e:
    log_to_unified(f"Configuration hub execution failed: {e}", "ERROR")
    print(f"❌ ERROR: Failed to launch the Configuration Hub. Details: {e}")
    raise

In [ ]:
#@title 🚀 **Cell 3: Lightning-Fast WebUI Installation & Launch** { display-mode: "form" }

import os
import sys
import json
import subprocess
import time
import threading
import re
import requests
import shutil
from pathlib import Path
from datetime import datetime
from IPython.display import HTML, display, clear_output
from concurrent.futures import ThreadPoolExecutor

def log_message(message, level="INFO"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{level}] {message}")

def run_lightning_fast_install(venv_pip_path, requirements_file_path, webui_name, cwd):
    """Lightning-fast installation using EXISTING venv and aria2c-cached packages"""
    print(f"\n⚡ Installing {webui_name} dependencies using existing venv...")

    cache_dir = Path(f'/tmp/webui_cache/{webui_name}')
    common_cache = Path('/tmp/webui_cache/common')

    start_time = time.time()

    # Phase 1: Install from aria2c-cached packages using existing venv
    cached_packages = list(cache_dir.glob('*.whl')) + list(common_cache.glob('*.whl'))
    if cached_packages:
        print(f"🚀 Phase 1: Installing {len(cached_packages)} pre-downloaded packages...")

        # Install cached packages using --find-links for efficiency
        cache_dirs = f'--find-links "{cache_dir}" --find-links "{common_cache}"'
        package_names = ' '.join([f'"{pkg.name}"' for pkg in cached_packages])
        cmd = f'"{venv_pip_path}" install {cache_dirs} --cache-dir /tmp/pip-cache --no-index --force-reinstall --no-deps {package_names}'

        try:
            result = subprocess.run(cmd, shell=True, cwd=cwd, capture_output=True, text=True, timeout=180)
            if result.returncode == 0:
                print(f"   ✅ Pre-downloaded packages installed successfully")
            else:
                print(f"   ⚠️ Some pre-downloaded packages had issues, continuing...")
        except Exception as e:
            print(f"   ⚠️ Cache installation error: {e}")

    # Phase 2: Install remaining from requirements using existing venv
    print(f"🔧 Phase 2: Installing remaining dependencies from requirements...")

    requirements_cmd = f'"{venv_pip_path}" install -r "{requirements_file_path}" --find-links "{cache_dir}" --find-links "{common_cache}" --cache-dir /tmp/pip-cache --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --progress-bar on --upgrade'

    try:
        process = subprocess.Popen(
            requirements_cmd, shell=True, cwd=cwd,
            stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
            text=True, bufsize=1
        )

        for line in iter(process.stdout.readline, ''):
            if any(keyword in line for keyword in [
                'Downloading', 'Installing', 'Successfully installed',
                'ERROR', 'Failed', 'Requirement already satisfied'
            ]):
                print(f"  > {line.strip()}")

        return_code = process.wait()
        duration = time.time() - start_time

        if return_code == 0:
            print(f"✅ Lightning-fast installation completed in {duration:.2f}s")
            return True
        else:
            print(f"⚠️ Installation had some issues but may still work")
            return True

    except Exception as e:
        print(f"❌ Installation error: {e}")
        return False

def install_webui_dependencies(webui_choice):
    """Install dependencies for the selected WebUI using EXISTING venv"""
    webui_path = Path(f'/content/{webui_choice}')
    venv_path = webui_path / 'venv'
    venv_pip = venv_path / 'bin' / 'pip'

    if not webui_path.exists():
        print(f"❌ {webui_choice} not found at {webui_path}")
        return False

    if not venv_path.exists():
        print(f"❌ Virtual environment not found for {webui_choice}")
        print("Please run Cell 1 first to create virtual environments")
        return False

    if not venv_pip.exists():
        print(f"❌ pip not found in virtual environment for {webui_choice}")
        return False

    print(f"\n{'='*80}")
    print(f"⚡ LIGHTNING-FAST INSTALLATION FOR {webui_choice}")
    print(f"📁 Using existing venv: {venv_path}")
    print(f"{'='*80}")

    # Find requirements file
    requirements_files = [
        webui_path / 'requirements_versions.txt',
        webui_path / 'requirements.txt'
    ]

    requirements_file = None
    for req_file in requirements_files:
        if req_file.exists():
            requirements_file = req_file
            break

    if not requirements_file:
        print(f"❌ No requirements file found for {webui_choice}")
        return False

    print(f"📋 Using requirements file: {requirements_file.name}")

    # Lightning-fast installation using existing venv
    success = run_lightning_fast_install(venv_pip, requirements_file, webui_choice, webui_path)

    if success:
        # Apply WebUI-specific optimizations using existing venv
        print(f"\n🔧 Applying {webui_choice}-specific optimizations...")

        webui_configs = {
            'A1111': [
                f'"{venv_pip}" install torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 --index-url https://download.pytorch.org/whl/cu121 --cache-dir /tmp/pip-cache',
                f'"{venv_pip}" install xformers==0.0.23.post1 --index-url https://download.pytorch.org/whl/cu121 --cache-dir /tmp/pip-cache'
            ],
            'Forge': [
                f'"{venv_pip}" install torch==2.3.1+cu121 torchvision==0.18.1+cu121 --index-url https://download.pytorch.org/whl/cu121 --cache-dir /tmp/pip-cache',
                f'"{venv_pip}" install xformers --cache-dir /tmp/pip-cache'
            ],
            'ComfyUI': [
                f'"{venv_pip}" install torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 --index-url https://download.pytorch.org/whl/cu121 --cache-dir /tmp/pip-cache',
                f'"{venv_pip}" install xformers==0.0.23.post1 --index-url https://download.pytorch.org/whl/cu121 --cache-dir /tmp/pip-cache'
            ],
            'InvokeAI': [
                f'"{venv_pip}" install torch==2.1.2+cu121 torchvision==0.16.2+cu121 --index-url https://download.pytorch.org/whl/cu121 --cache-dir /tmp/pip-cache'
            ],
            'FooocusUI': [
                f'"{venv_pip}" install torch==2.1.2+cu121 torchvision==0.16.2+cu121 --index-url https://download.pytorch.org/whl/cu121 --cache-dir /tmp/pip-cache'
            ]
        }

        commands = webui_configs.get(webui_choice, [])
        for i, cmd in enumerate(commands, 1):
            component = 'PyTorch' if 'torch' in cmd else 'xformers'
            print(f"🔧 Installing {component}...")
            try:
                result = subprocess.run(cmd, shell=True, cwd=webui_path, capture_output=True, text=True, timeout=300)
                if result.returncode == 0:
                    print(f"   ✅ {component} installed successfully")
                else:
                    print(f"   ⚠️ {component} had issues (may still work): {result.stderr[:100] if result.stderr else 'Unknown error'}")
            except Exception as e:
                print(f"   ⚠️ {component} installation error: {e}")

    # Apply matplotlib fix
    apply_matplotlib_fix(webui_path)

    return success

def apply_matplotlib_fix(webui_path):
    """Apply matplotlib backend fix to launch files - CORRECTED VERSION"""
    print("🔧 Applying matplotlib backend fix...")

    launch_files = ['launch.py', 'webui.py', 'main.py']

    fix_code = (
        "# Trinity matplotlib backend fix\n"
        "import os\n"
        "os.environ['MPLBACKEND'] = 'Agg'\n"
        "try:\n"
        "    import matplotlib\n"
        "    matplotlib.use('Agg')\n"
        "except ImportError:\n"
        "    pass\n"
    )

    for launch_file in launch_files:
        file_path = webui_path / launch_file
        if file_path.exists():
            try:
                content = file_path.read_text(encoding='utf-8')

                # Skip if fix already applied
                if "matplotlib.use('Agg')" in content:
                    print(f"   ✅ Matplotlib fix already present in {launch_file}")
                    continue

                lines = content.splitlines()
                insert_position = 0

                # Find the correct insertion point
                for i, line in enumerate(lines):
                    stripped = line.strip()

                    # Skip shebang
                    if stripped.startswith('#!'):
                        insert_position = i + 1
                        continue

                    # Skip comments and docstrings at the top
                    if (stripped.startswith('#') or
                        stripped.startswith('"""') or
                        stripped.startswith("'''") or
                        stripped == ""):
                        insert_position = i + 1
                        continue

                    # Handle future imports - they MUST come first
                    if stripped.startswith('from __future__ import'):
                        insert_position = i + 1
                        continue

                    # Stop at first non-future import or code
                    break

                # Insert the matplotlib fix
                fix_lines = fix_code.strip().split('\n')

                # Add empty line before fix if needed
                if insert_position < len(lines) and lines[insert_position].strip() != "":
                    fix_lines.append("")

                # Insert fix lines
                for i, fix_line in enumerate(fix_lines):
                    lines.insert(insert_position + i, fix_line)

                # Write back to file
                new_content = '\n'.join(lines)
                file_path.write_text(new_content, encoding='utf-8')
                print(f"   ✅ Applied matplotlib fix to {launch_file} at position {insert_position + 1}")

            except Exception as e:
                print(f"   ⚠️ Error applying fix to {launch_file}: {e}")
                # If fix fails, try a safer approach - just remove any existing bad fix
                try:
                    content = file_path.read_text(encoding='utf-8')
                    # Remove any existing Trinity matplotlib fix
                    lines = content.splitlines()
                    clean_lines = []
                    skip_next = False

                    for line in lines:
                        if "Trinity matplotlib backend fix" in line:
                            skip_next = True
                            continue
                        if skip_next and ("import os" in line or "matplotlib" in line or line.strip() == ""):
                            continue
                        if skip_next and line.strip() and not any(x in line for x in ["import", "os.environ", "matplotlib", "except", "pass"]):
                            skip_next = False

                        if not skip_next:
                            clean_lines.append(line)

                    file_path.write_text('\n'.join(clean_lines), encoding='utf-8')
                    print(f"   🔧 Cleaned up {launch_file} (removed bad matplotlib fix)")
                except:
                    print(f"   ❌ Could not fix {launch_file}")

def download_file_with_aria2c(url, output_path, filename):
    """Download a single file using aria2c with acceleration"""
    try:
        output_path.mkdir(parents=True, exist_ok=True)

        cmd = [
            'aria2c',
            '--max-connection-per-server=16',
            '--split=16',
            '--min-split-size=1M',
            '--continue=true',
            '--retry-wait=3',
            '--max-tries=5',
            '--dir', str(output_path),
            '--out', filename,
            url
        ]

        result = subprocess.run(cmd, capture_output=True, text=True, timeout=600)

        file_path = output_path / filename
        if result.returncode == 0 and file_path.exists() and file_path.stat().st_size > 0:
            return True, f"Downloaded {filename} successfully"
        else:
            return False, f"Download failed: {result.stderr[:200] if result.stderr else 'Unknown error'}"

    except subprocess.TimeoutExpired:
        return False, f"Download timed out for {filename}"
    except Exception as e:
        return False, f"Download error for {filename}: {e}"

def get_model_data_for_version(is_xl=False):
    """Get model data based on SD version"""
    try:
        project_root = Path('/content/TrinityUI')
        if is_xl:
            data_file = project_root / 'scripts' / '_xl-models-data.py'
        else:
            data_file = project_root / 'scripts' / '_models-data.py'

        if not data_file.exists():
            print(f"❌ Model data file not found: {data_file}")
            return {}, {}, {}, {}

        # Read and execute the Python file to get the data
        with open(data_file, 'r', encoding='utf-8') as f:
            content = f.read()

        # Create a namespace to execute the file
        namespace = {}
        exec(content, namespace)

        model_list = namespace.get('model_list', {})
        vae_list = namespace.get('vae_list', {})
        controlnet_list = namespace.get('controlnet_list', {})
        lora_list = namespace.get('lora_list', {})

        print(f"📊 Loaded data: {len(model_list)} models, {len(vae_list)} VAEs, {len(controlnet_list)} ControlNets, {len(lora_list)} LoRAs")

        return model_list, vae_list, controlnet_list, lora_list

    except Exception as e:
        print(f"❌ Error loading model data: {e}")
        return {}, {}, {}, {}

def download_selected_assets(config):
    """Download ONLY the selected models, VAEs, ControlNets, and LoRAs from config"""
    webui_choice = config.get('webui_choice', 'A1111')
    is_xl = config.get('sd_version') == 'SDXL'

    selected_models = config.get('selected_models', [])
    selected_vaes = config.get('selected_vaes', [])
    selected_controlnets = config.get('selected_controlnets', [])
    selected_loras = config.get('selected_loras', [])

    print(f"🎯 Downloading ONLY selected assets for {webui_choice}")
    print(f"📋 Your selections:")
    print(f"   Models: {selected_models}")
    print(f"   VAEs: {selected_vaes}")
    print(f"   ControlNets: {selected_controlnets}")
    print(f"   LoRAs: {selected_loras}")

    if not any([selected_models, selected_vaes, selected_controlnets, selected_loras]):
        print("✅ No assets selected for download")
        return True

    # Get model data for the correct SD version
    model_list, vae_list, controlnet_list, lora_list = get_model_data_for_version(is_xl)

    if not any([model_list, vae_list, controlnet_list, lora_list]):
        print("❌ Failed to load model data files")
        return False

    # Setup download directories for the selected WebUI
    webui_path = Path(f'/content/{webui_choice}')

    # Different WebUIs have different directory structures
    if webui_choice == 'ComfyUI':
        models_dir = webui_path / 'models' / 'checkpoints'
        vae_dir = webui_path / 'models' / 'vae'
        controlnet_dir = webui_path / 'models' / 'controlnet'
        lora_dir = webui_path / 'models' / 'loras'
    else:  # A1111, Forge, etc.
        models_dir = webui_path / 'models' / 'Stable-diffusion'
        vae_dir = webui_path / 'models' / 'VAE'
        controlnet_dir = webui_path / 'models' / 'ControlNet'
        lora_dir = webui_path / 'models' / 'Lora'

    download_tasks = []

    # Process ONLY selected models
    for model_name in selected_models:
        if model_name in model_list:
            print(f"📦 Found selected model: {model_name}")
            for item in model_list[model_name]:
                url = item.get('url', '')
                filename = item.get('name', url.split('/')[-1])
                if url and filename:
                    download_tasks.append({
                        'url': url,
                        'path': models_dir,
                        'filename': filename,
                        'type': 'Model',
                        'selection': model_name
                    })
        else:
            print(f"⚠️ Selected model not found in data: {model_name}")

    # Process ONLY selected VAEs
    for vae_name in selected_vaes:
        if vae_name in vae_list:
            print(f"📦 Found selected VAE: {vae_name}")
            for item in vae_list[vae_name]:
                url = item.get('url', '')
                filename = item.get('name', url.split('/')[-1])
                if url and filename:
                    download_tasks.append({
                        'url': url,
                        'path': vae_dir,
                        'filename': filename,
                        'type': 'VAE',
                        'selection': vae_name
                    })
        else:
            print(f"⚠️ Selected VAE not found in data: {vae_name}")

    # Process ONLY selected ControlNets
    for controlnet_name in selected_controlnets:
        if controlnet_name in controlnet_list:
            print(f"📦 Found selected ControlNet: {controlnet_name}")
            for item in controlnet_list[controlnet_name]:
                url = item.get('url', '')
                filename = item.get('name', url.split('/')[-1])
                if url and filename:
                    download_tasks.append({
                        'url': url,
                        'path': controlnet_dir,
                        'filename': filename,
                        'type': 'ControlNet',
                        'selection': controlnet_name
                    })
        else:
            print(f"⚠️ Selected ControlNet not found in data: {controlnet_name}")

    # Process ONLY selected LoRAs
    for lora_name in selected_loras:
        if lora_name in lora_list:
            print(f"📦 Found selected LoRA: {lora_name}")
            for item in lora_list[lora_name]:
                url = item.get('url', '')
                filename = item.get('name', url.split('/')[-1])
                if url and filename:
                    download_tasks.append({
                        'url': url,
                        'path': lora_dir,
                        'filename': filename,
                        'type': 'LoRA',
                        'selection': lora_name
                    })
        else:
            print(f"⚠️ Selected LoRA not found in data: {lora_name}")

    if not download_tasks:
        print("❌ No valid download URLs found for your selections")
        return False

    print(f"\n📥 Starting download of {len(download_tasks)} files from your selections...")

    # Download with aria2c acceleration
    success_count = 0
    total_count = len(download_tasks)

    for i, task in enumerate(download_tasks, 1):
        print(f"🔄 [{i}/{total_count}] Downloading {task['type']} from '{task['selection']}': {task['filename']}")

        # Check if file already exists
        file_path = task['path'] / task['filename']
        if file_path.exists() and file_path.stat().st_size > 1024*1024:  # > 1MB
            print(f"   ✅ Already exists: {task['filename']}")
            success_count += 1
            continue

        success, message = download_file_with_aria2c(
            task['url'],
            task['path'],
            task['filename']
        )

        if success:
            success_count += 1
            print(f"   ✅ {message}")
        else:
            print(f"   ❌ {message}")

    print(f"\n📊 Download Summary: {success_count}/{total_count} selected assets downloaded successfully")

    if success_count > 0:
        print("✅ Selected asset download completed!")
        return True
    else:
        print("⚠️ Some selected assets failed to download, but WebUI will still work")
        return True

def run_asset_download(config):
    """Download models and assets for the selected WebUI"""
    webui_choice = config.get('webui_choice', 'A1111')
    selected_models = config.get('selected_models', [])
    selected_vaes = config.get('selected_vaes', [])
    selected_controlnets = config.get('selected_controlnets', [])
    selected_loras = config.get('selected_loras', [])

    print(f"\n{'='*80}")
    print(f"🎁 DOWNLOADING SELECTED ASSETS FOR {webui_choice}")
    print(f"{'='*80}")

    if not any([selected_models, selected_vaes, selected_controlnets, selected_loras]):
        print("✅ No assets selected for download. Ready for launch!")
        return True

    print(f"📦 You selected:")
    print(f"   Models: {len(selected_models)} - {selected_models}")
    print(f"   VAEs: {len(selected_vaes)} - {selected_vaes}")
    print(f"   ControlNets: {len(selected_controlnets)} - {selected_controlnets}")
    print(f"   LoRAs: {len(selected_loras)} - {selected_loras}")

    try:
        download_success = download_selected_assets(config)

        if download_success:
            print("✅ Selected asset download process completed!")
            return True
        else:
            print("⚠️ Some selected assets failed to download, but WebUI is ready!")
            return True

    except Exception as e:
        print(f"⚠️ Selected asset download failed: {e}")
        print("Assets will be downloaded automatically on first use.")
        return True

def highlight_gradio_link(line):
    """Detects and highlights Gradio links in the output"""
    patterns = [
        r'(https?://[a-z0-9]+\.gradio\.live)',
        r'(https?://[a-z0-9\-]+\.gradio\.app)',
        r'(Running on public URL: https?://[^\s]+)',
        r'(Public WebUI URL: https?://[^\s]+)'
    ]

    for pattern in patterns:
        match = re.search(pattern, line)
        if match:
            url = match.group(1)
            if 'http' not in url:
                url = re.search(r'(https?://[^\s]+)', url).group(1)

            print("\n" + "="*80)
            print(f"🌐 GRADIO LINK DETECTED: {url}")
            print("="*80 + "\n")

            display(HTML(f'''
            <div style="padding: 20px; background-color: #d1e7dd; border-radius: 10px; margin: 20px 0; border-left: 5px solid #198754;">
                <h2 style="color: #198754; margin-top: 0;">🎉 WebUI is Ready!</h2>
                <p style="margin: 15px 0; font-size: 16px;">Your WebUI is now running and accessible via the public URL below:</p>
                <div style="text-align: center; margin: 20px 0;">
                    <a href="{url}" target="_blank" style="
                        font-size: 18px;
                        font-weight: bold;
                        color: white;
                        text-decoration: none;
                        padding: 15px 30px;
                        background-color: #198754;
                        border-radius: 8px;
                        display: inline-block;
                        box-shadow: 0 4px 8px rgba(0,0,0,0.2);
                    ">🚀 Open WebUI</a>
                </div>
                <p style="margin: 10px 0; font-size: 14px; color: #495057;">
                    <strong>URL:</strong> <code style="background-color: #f8f9fa; padding: 2px 4px; border-radius: 3px;">{url}</code>
                </p>
            </div>
            '''))
            return True
    return False

def launch_webui_with_detection(webui_choice, max_wait_time=300):
    """Launch WebUI using EXISTING venv and detect Gradio URL"""

    # Load config to get launch arguments
    config_path = Path('/content/TrinityUI/trinity_config.json')
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        custom_args = config.get('custom_args', '')
        tunnel_choice = config.get('tunnel_choice', 'Gradio')
    except:
        custom_args = '--share --xformers'
        tunnel_choice = 'Gradio'

    # FORCE --share for Gradio regardless of config
    args_list = custom_args.split() if custom_args else []
    if '--share' not in args_list:
        args_list.append('--share')

    # Ensure other useful arguments
    if '--xformers' not in args_list and webui_choice in ['A1111', 'Forge']:
        args_list.append('--xformers')

    if '--enable-insecure-extension-access' not in args_list:
        args_list.append('--enable-insecure-extension-access')

    webui_path = Path(f'/content/{webui_choice}')
    venv_python = webui_path / 'venv' / 'bin' / 'python'

    if not venv_python.exists():
        print(f"❌ Python executable not found in venv: {venv_python}")
        print("Please run Cell 1 first to create virtual environments")
        return None

    args = [str(venv_python), 'launch.py'] + args_list

    print(f"🚀 Launching {webui_choice} using existing venv")
    print(f"📁 Venv path: {webui_path / 'venv'}")
    print(f"🔧 Arguments: {' '.join(args_list)}")
    print("📡 Waiting for Gradio public URL...")

    # Set environment with matplotlib fix
    env = os.environ.copy()
    env["MPLBACKEND"] = "Agg"
    env["PYTHONUNBUFFERED"] = "1"

    process = subprocess.Popen(
        args,
        cwd=webui_path,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        env=env
    )

    start_time = time.time()
    url_found = False

    try:
        while time.time() - start_time < max_wait_time:
            line = process.stdout.readline()
            if not line:
                if process.poll() is not None:
                    print("❌ WebUI process ended unexpectedly")
                    break
                time.sleep(0.1)
                continue

            # Print important status messages
            if any(keyword in line.lower() for keyword in [
                'loading', 'model loaded', 'startup time', 'applying', 'running on',
                'local url', 'public url', 'loading config', 'initializing', 'traceback',
                'error', 'failed', 'exception'
            ]):
                print(f"   {line.strip()}")

            # Look for Gradio URL
            if any(keyword in line for keyword in ["Running on public URL:", "gradio.live", "gradio.app"]):
                url_found = highlight_gradio_link(line)
                if url_found:
                    print("✅ WebUI is now accessible via the public URL above!")
                    break

        if not url_found:
            print(f"❌ No Gradio URL found within {max_wait_time} seconds")
            print("💡 The WebUI may still be starting. Try these commands:")
            print(f"!cd /content/{webui_choice} && MPLBACKEND=Agg /content/{webui_choice}/venv/bin/python launch.py --share --xformers --enable-insecure-extension-access")
            process.terminate()
            return None

        # Keep process running
        try:
            process.wait()
        except KeyboardInterrupt:
            print("\n✅ WebUI shutdown requested by user.")
            process.terminate()

    except KeyboardInterrupt:
        print("WebUI launch interrupted by user")
        process.terminate()
        return None

# Infrastructure Check
PROJECT_ROOT = Path('/content/TrinityUI')
CONFIG_PATH = PROJECT_ROOT / "trinity_config.json"

# Force matplotlib backend immediately
os.environ["MPLBACKEND"] = "Agg"

if not PROJECT_ROOT.exists():
    log_message("PROJECT_ROOT not found. Please run Cell 1 first.", "ERROR")
    raise RuntimeError("PROJECT_ROOT not found. Please run Cell 1 first.")

if not CONFIG_PATH.exists():
    log_message(f"Configuration file not found at {CONFIG_PATH}", "ERROR")
    log_message("Please run Cell 2 and complete the configuration process.", "ERROR")
    raise FileNotFoundError(f"Configuration file not found at {CONFIG_PATH}")

try:
    with open(CONFIG_PATH, 'r') as f:
        config = json.load(f)
    webui_choice = config.get('webui_choice', 'Unknown')
    log_message(f"Configuration loaded for {webui_choice}", "SUCCESS")
except Exception as e:
    log_message(f"Failed to load config file: {e}", "ERROR")
    raise

print("="*80)
print("⚡ TRINITY LIGHTNING-FAST WEBUI LAUNCH")
print("="*80)

# Check if we need to install dependencies first
webui_path = Path(f'/content/{webui_choice}')
venv_path = webui_path / 'venv'
requirements_installed = False

if venv_path.exists():
    # Check if key packages are installed in the existing venv
    try:
        venv_python = venv_path / 'bin' / 'python'
        result = subprocess.run([str(venv_python), '-c', 'import torch; print("OK")'],
                              capture_output=True, text=True, timeout=10)
        requirements_installed = result.returncode == 0
        if requirements_installed:
            print(f"✅ Dependencies already installed in existing venv for {webui_choice}")
    except:
        requirements_installed = False
else:
    print(f"❌ No virtual environment found for {webui_choice}")
    print("Please run Cell 1 first to create virtual environments")
    raise RuntimeError("Virtual environment not found")

if not requirements_installed:
    print(f"🔧 Installing dependencies for {webui_choice} using existing venv...")
    deps_success = install_webui_dependencies(webui_choice)

    if not deps_success:
        print(f"❌ Failed to install {webui_choice} dependencies")
        print("Try running Cell 2 again to reconfigure.")
        raise RuntimeError("Dependency installation failed")

    print(f"✅ {webui_choice} dependencies installed successfully!")

# Download ONLY selected assets
print(f"\n🎯 PHASE 2: Download ONLY your selected assets...")
assets_success = run_asset_download(config)

if assets_success:
    print(f"\n✅ Selected asset download process completed!")
else:
    print(f"\n⚠️ Selected asset download had issues, but WebUI is ready!")

# Launch the WebUI using existing venv
print(f"\n🚀 Launching {webui_choice} using existing virtual environment...")
launch_webui_with_detection(webui_choice)

In [ ]:
!cd /content/A1111 && MPLBACKEND=Agg /content/A1111/venv/bin/python launch.py --share --xformers --enable-insecure-extension-access

In [ ]:
#@title 🚀 **Direct Launch with Matplotlib Fix** { display-mode: "form" }

import os
import subprocess
import time
from pathlib import Path

# Force matplotlib backend before any imports
os.environ["MPLBACKEND"] = "Agg"

def launch_webui_direct():
    webui_path = Path('/content/A1111')
    venv_python = webui_path / 'venv' / 'bin' / 'python'

    # Build command
    cmd = [
        str(venv_python),
        "launch.py",
        "--share",
        "--xformers",
        "--api",
        "--enable-insecure-extension-access"
    ]

    print("🚀 Launching A1111 with matplotlib backend fix...")
    print(f"Command: {' '.join(cmd[1:])}")

    # Set environment with matplotlib fix
    env = os.environ.copy()
    env["MPLBACKEND"] = "Agg"
    env["PYTHONUNBUFFERED"] = "1"

    # Launch process
    process = subprocess.Popen(
        cmd,
        cwd=webui_path,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        env=env
    )

    print("📡 Waiting for Gradio URL...")

    try:
        for line in iter(process.stdout.readline, ''):
            print(line.rstrip())

            # Look for Gradio URL
            if "Running on public URL:" in line:
                import re
                url_match = re.search(r'https://[a-z0-9]+\.gradio\.live', line)
                if url_match:
                    gradio_url = url_match.group(0)
                    print(f"\n🎉 SUCCESS! WebUI is ready at: {gradio_url}")
                    print("📌 Stop this cell to shutdown the WebUI")

        process.wait()

    except KeyboardInterrupt:
        print("\n✅ Shutting down WebUI...")
        process.terminate()
        process.wait()

# Check if A1111 exists
if not Path('/content/A1111').exists():
    print("❌ A1111 not found. Please run Cell 1 first.")
else:
    launch_webui_direct()